In [2]:
from dash import Dash, dcc, html
import pandas as pd
from crud import CRUD 
import plotly.express as px

In [3]:

username = "aacuser"
password = "jannatuldbuser"  
crud = CRUD(username, password)

app = dash.Dash(__name__)
app.layout = html.Div(children=[
    html.H1("Grazioso Salvare Dog Rescue Dashboard"),
    html.A(html.Img(src="https://learn.snhu.edu/content/enforced/1698584-CS-340-10940.202481-1/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=ScQoRYLN9OTGv4T9RbCyYwie6&ou=1332057&ou=1698584", height="100px"), href="http://www.snhu.edu"),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'block'}
    ),

   
    html.Div(id='data-table'),

    
    html.Div(id='map-id'),

    
    html.Div(id='secondary-chart')
])
@app.callback(
    Output('data-table', 'children'),
    Input('rescue-type', 'value')
)
def update_data_table(selected_type):
   
    data = crud.retrieve_all()
    if selected_type == 'Water':
        filtered_data = data[data['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])]
    elif selected_type == 'Mountain':
        filtered_data = data[data['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])]
    elif selected_type == 'Disaster':
        filtered_data = data[data['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])]
    else:  
        filtered_data = data
  
    return dcc.DataTable(
        data=filtered_data.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single'
    )
@app.callback(
    Output('map-id', 'children'),
    Input('data-table', 'selected_rows')
)
def update_map(selected_rows):
    if not selected_rows:
        return html.Div()  

    selected_row_index = selected_rows[0]
  
    data = crud.retrieve_all()
    selected_data = data.iloc[selected_row_index]

    
    fig = px.scatter_mapbox(
        data_frame=data,
        lat='latitude',  
        lon='longitude',  
        hover_name='name',  
        title='Geolocation Chart'
    )
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return dcc.Graph(figure=fig)


@app.callback(
    Output('secondary-chart', 'children'),
    Input('rescue-type', 'value')
)
def update_secondary_chart(selected_type):
    data = crud.retrieve_all()
    if selected_type == 'Water':
        filtered_data = data[data['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])]
    elif selected_type == 'Mountain':
        filtered_data = data[data['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])]
    elif selected_type == 'Disaster':
        filtered_data = data[data['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])]
    else:
        filtered_data = data

    fig = px.pie(filtered_data, names='breed', title='Dog Breeds Distribution')
    return dcc.Graph(figure=fig)


if __name__ == '__main__':
    app.run_server(debug=True)


TypeError: CRUD.__init__() takes 2 positional arguments but 3 were given